## Gemma 장점

- 다양한 모델 제공: Gemma는 Gemma 2B와 Gemma 7B 두 가지 유형으로 제공, 각 유형은 pre-trained 및 instruction-tuned 된 두가지 타입의 모델 제공
- 책임감 있는 AI 툴킷: Gemma를 활용하여 안전한 AI 애플리케이션을 제작할 수 있는 가이드와 필수 도구를 제공하는 새로운 책임감 있는 생성형 AI 툴킷이 함께 제공
- 프레임워크 지원: Kera JAX, PyTorch, TensorFlow와 같은 주요 프레임워크에서 추론 및 SFT을 위한 툴체인을 제공
- 다양한 하드웨어 지원: NVIDIA GPU와 구글 클라우드 TPU 등에 최적화

## Gemma를 위한 라이브러리

- pip install -q -U transformers==4.38.2
- pip install -q -U datasets==2.18.0
- pip install -q -U bitsandbytes==0.42.0
- pip install -q -U peft==0.9.0
- pip install -q -U trl==0.7.11
- pip install -q -U accelerate==0.27.2

## Gemma 기타

- 트랜스포머 디코더 쪽이다.

In [1]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from huggingface_hub import notebook_login
from datasets import load_dataset
torch.cuda.empty_cache()

## 1. 허깅페이스 로그인

In [2]:
notebook_login()

## 2. 데이터셋 로드

In [3]:
# TEST_NAME = "beomi-gemma-ko-2b-01"
TEST_NAME = "google-gemma-2b-it-02"

import os
import platform

# 데이터 관련
os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
DATA_PATH = PRE_PATH + "data/" # 대회에서 제공한 데이터
# TRAIN_PATH = DATA_PATH + "train_kkh_new.csv"
TRAIN_PATH = DATA_PATH + "train_hdj.csv"
VALID_PATH = DATA_PATH + "dev_kkh_new.csv"
TEST_PATH = DATA_PATH + "test.csv"
PREDICTION_PATH = PRE_PATH + "prediction/" # 최종 예측 값
SUBMIT_PATH = PREDICTION_PATH + "submission_" + TEST_NAME + ".csv"

# BASE_MODEL = "beomi/gemma-ko-2b"
BASE_MODEL = "google/gemma-2b-it"
# BASE_MODEL = "/kkh/gemma_model/gemma-2b-it-sum-ko-10epc"
ADAPTER_MODEL = PRE_PATH + "gemma_model/adapter/" + TEST_NAME
FINETUNE_MODEL = PRE_PATH + "gemma_model/finetune/" + TEST_NAME

DATASET_TRAIN = load_dataset('csv', data_files={'train': TRAIN_PATH})
DATASET_TEST = load_dataset('csv', data_files={'test': TEST_PATH})
TRAIN_DATA = DATASET_TRAIN['train']

In [4]:
# 데이터셋 확인
print(TRAIN_DATA[0])
print('=============================')
print(f"train 대화문:\n{TRAIN_DATA[0]['dialogue']}")
print('=============================')
print(f"train 요약문:\n{TRAIN_DATA[0]['summary']}")
print('=============================')
print(f"test 대화문:\n{TRAIN_DATA[0]['dialogue']}")

{'Unnamed: 0': 0, 'fname': 'train_0', 'dialogue': '#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?\n#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.\n#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.\n#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?\n#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.\n#Person2#: 알겠습니다.\n#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?\n#Person2#: 네.\n#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. \n#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.\n#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.\n#Person2#: 알겠습니다, 감사합니다, 의사선생님.', 'summary': '스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.', 'topic': '건강검진 받기'}
train 대화문:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질

## 3.1 모델 로드

In [5]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# 모델 자체를 몇 비트로 로딩할 것인지를 정하는 것이다.
# 32, 16, 8에 비해 4비트는 메모리를 적게 사용하면서 속도가 빠르지만, 정확도가 떨어진다.
# 서버 자원이 적을 경우 주로 사용한다.
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

# device_map="auto"는 데이터를 CPU 또는 GPU로 자동 연결해주는 기능이다. GPU가 여러개 이면, 자동 분산도 해준다.
# quantization_config=quantization_config 에 대한 설명은 위 코드에 있다.
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=quantization_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 3.2 Gemma-it의 프롬프트 형식

In [6]:
ex = TRAIN_DATA[0]['dialogue']
messages = [
    {
        "role": "user",
        "content": "다음 글을 해설자가 설명하듯이 요약해주세요.:\n\n{}".format(ex)
    }
]

# tokenize: 문장을 토큰 단위를 자를 것인지 지정한다.
# add_generation_prompt: 다음 발화자가 말할 수 있도록, 프롬프트를 적어준다. 예를 들어, user와 model이 대화를 하는 중이라면, user 발화 끝 부분에 "<start_of_turn>model" 과 같이 붙여준다.
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
prompt

'<bos><start_of_turn>user\n다음 글을 해설자가 설명하듯이 요약해주세요.:\n\n#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?\n#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.\n#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.\n#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?\n#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.\n#Person2#: 알겠습니다.\n#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?\n#Person2#: 네.\n#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. \n#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.\n#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.\n#Person2#: 알겠습니다, 감사합니다, 의사선생님.<end_of_turn>\n<start_of_turn>model\n'

## 3.3 Gemma-it 추론

In [7]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.01,
    top_k=10,
    top_p=0.2
)

# do_sample=True: 생성할 때 확률적인 샘플링을 사용하여 보다 다양한 출력을 생성합니다. False로 설정 시 가장 높은 확률의 토큰을 선택합니다.
# temperature=0.5: 모델의 출력 확률 분포를 조정합니다. 값이 낮을수록 보수적이고, 높을수록 다양하게 생성합니다.
# top_k=50: 모델이 선택할 수 있는 상위 50개의 토큰 중 하나를 샘플링합니다.
# top_p=0.95: 확률 분포의 상위 95%에 해당하는 토큰을 선택해 생성하는 방식입니다.

/opt/conda/envs/nlp/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [8]:
print(outputs[0]["generated_text"][len(prompt):])

**해설:**

이 문의에서 주요 내용은 다음과 같습니다.

- 건강검진은 5년 동안 받아야 할 수 있는 기회가 있다고 얘기합니다.
- 심각한 질병을 피하기 위해서는 이를 조기에 발견하는 것이 중요합니다.
- 담배는 폐암과 심장병의 주요 원인이므로, 끊임없이 피우는 것이 중요합니다.
- 의사에게 의문이 있으면 더 많은 정보를 드리겠다고 약속합니다.


## 4.1 학습용 프롬프트 조정

In [9]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['dialogue'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{}

<end_of_turn>
<start_of_turn>model

{}

<end_of_turn><eos>""".format(example['dialogue'][i], example['summary'][i]))
    return prompt_list

In [10]:
prompt_ex_one = generate_prompt(TRAIN_DATA[:1])[0]
print(prompt_ex_one)

<bos><start_of_turn>user
다음 글을 요약해주세요:

#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.

<end_of_turn>
<start_of_turn>model

스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.

<end_of_turn><eos>


## 4.2 QLoRA 설정

In [11]:
lora_config = LoraConfig(
    r=4,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

# r (8로 증가):
# 장점: 더 복잡한 표현이 가능해져 성능이 개선될 수 있음.
# 단점: 메모리 사용량이 늘어나고, 계산 비용이 증가.
# lora_alpha (16로 증가):
# 장점: LoRA의 출력 강도가 커져서, 모델 성능이 좋아질 가능성.
# 단점: 오버피팅 가능성 증가, 훈련 시간이 더 길어질 수 있음.
# lora_dropout (0.1로 증가):
# 장점: 과적합 방지에 도움이 되어, 모델의 일반화 성능이 향상.
# 단점: 너무 큰 드롭아웃 값은 학습 성능 저하로 이어질 수 있음.
# target_modules: LoRA를 적용할 모델 레이어들입니다. 주로 트랜스포머의 어텐션 및 피드포워드 관련 레이어에 적용됩니다.
# target_modules: LoRA가 적용되는 모듈의 범위를 줄이면 연산량 감소, 특정 성능 유지. 하지만, 너무 적은 모듈에 적용하면 학습 성능 저하.
# ???task_type="CAUSAL_LM": 이 설정은 모델이 Causal Language Modeling 작업에 적합하도록 맞춰집니다.

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# load_in_4bit=True:
# 장점: 메모리 사용량이 줄어들어 대형 모델을 작은 GPU에서도 실행 가능.
# 단점: 양자화로 인해 성능이 약간 떨어질 수 있음.

# bnb_4bit_quant_type="fp4" (fp4로 변경):
# 장점: 더 높은 계산 정밀도를 제공, 양자화로 인한 성능 손실을 줄임.
# 단점: 메모리 사용량이 약간 늘어날 수 있음.

# bnb_4bit_compute_dtype=torch.bfloat16 (bfloat16으로 변경):
# 장점: float16보다 더 넓은 범위를 처리해, 안정성 향상.
# 단점: 연산 속도가 약간 느려질 수 있음.

In [12]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

# 오른쪽에만 패딩을 넣는 이유는 주로 Causal Language Modeling (예측 기반 언어 모델)에서 모델이 입력의 왼쪽에서 오른쪽으로 순차적으로 문맥을 처리하기 때문입니다. 오른쪽에 패딩을 넣으면 패딩된 부분이 미래의 토큰처럼 처리되지 않으며, 자연스러운 문맥 예측이 가능합니다. 특히 GPT 계열 모델처럼 시퀀스 생성 방식의 모델에서 이러한 방식은 매우 중요합니다.
# 왼쪽에 패딩을 넣으면 패딩이 문맥에 영향을 줄 수 있어 예측 성능이 떨어질 수 있습니다.

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 4.3 Trainer 실행

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=TRAIN_DATA,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 2,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_ratio=1,
        learning_rate=2e-4,
        fp16=True,
        fp16_full_eval=True,  # kkh 추가함
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

# model=model: 미리 학습된 모델을 트레이너에 전달합니다.
# train_dataset=TRAIN_DATA: 훈련에 사용할 데이터셋.
# max_seq_length=512: 입력 시퀀스의 최대 길이.
# output_dir="outputs": 훈련 후 결과를 저장할 디렉토리.
# num_train_epochs=1: 전체 데이터셋에 대해 1번의 에포크로 훈련.
# max_steps=100: 최대 100 스텝까지 훈련.
# per_device_train_batch_size=1: 장치당 배치 크기.
# gradient_accumulation_steps=4: 4 스텝마다 경사도를 누적.
# optim="paged_adamw_8bit": 8비트 AdamW 옵티마이저 사용.
# warmup_ratio=0.02: 학습률 워밍업 비율.
# learning_rate=2e-4: 학습률.
# fp16=True: 16비트 부동소수점 연산 사용.
# fp16_full_eval=True: 평가 시에도 FP16 사용.
# logging_steps=100: 100 스텝마다 로그 기록.
# push_to_hub=False: 허브로 푸시하지 않음.
# report_to='none': 로깅을 사용하지 않음.
# peft_config=lora_config: LoRA(매개변수 효율적인 미세 조정) 설정.
# formatting_func=generate_prompt: 데이터셋 전처리용 함수.

Map:   0%|          | 0/12447 [00:00<?, ? examples/s]

/opt/conda/envs/nlp/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [16]:
best_train_loss = float('inf')  # 가장 낮은 train loss를 추적하기 위한 변수
for epoch in range(int(trainer.args.num_train_epochs)):  # 에포크 루프
    train_output = trainer.train()  # 한 에포크의 학습 진행
    train_loss = train_output.training_loss  # 현재 에포크의 train loss

    # 가장 낮은 train loss가 발견되면 모델 저장
    if train_loss < best_train_loss:
        best_train_loss = train_loss
        trainer.model.save_pretrained(ADAPTER_MODEL + "_" + str(best_train_loss))  # 가장 낮은 train loss 시점의 모델 저장
        print(f"Best model saved with train loss: {best_train_loss}")

Step,Training Loss
100,1.271100
200,1.272100
300,1.238500
400,1.231000
500,1.224600
600,1.215400
700,1.226600
800,1.224000
900,1.216600


KeyboardInterrupt: 

In [17]:
trainer.model.save_pretrained(ADAPTER_MODEL + "_1.2166")

## 4.4 Finetuned Model 저장

In [18]:
ADAPTER_MODEL = ADAPTER_MODEL + "_1.2166" # 최고의 모델을 폴더에서 찾아서 적어야 함

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained(FINETUNE_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 5.1 Fine-tuned 모델 로드

In [19]:
finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 5.2 Fine-tuned 모델 추론

In [20]:
import csv

# 요약을 저장할 리스트
summaries = []

# 데이터셋 전체 요약 처리
for i, ex in enumerate(DATASET_TEST['test']['dialogue']):
    # 각 대화마다 메시지 포맷 설정
    messages = [
        {
            "role": "user",
            "content": "다음 글을 요약해주세요:\n\n{}".format(ex)
        }
    ]
    
    # 프롬프트 생성
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # 요약 파이프라인
    pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)
    
    # 모델을 통해 요약 생성
    outputs = pipe_finetuned(
        prompt,
        do_sample=True,
        temperature=0.01,
        top_k=20,
        top_p=0.92,
        add_special_tokens=True
    )
    
    # 요약 텍스트 저장
    summary = outputs[0]["generated_text"][len(prompt):].strip()
    
    # 요약에 콤마가 있으면 쌍따옴표로 감싸기
    if ',' in summary:
        summary = f'"{summary}"'
    
    # summaries 리스트에 저장
    summaries.append([f"test_{i}", summary])
    
    # 요약 출력
    # test 문제에 466번은 원래 존재하지 않음
    if i >= 466:
        i += 1
    print(f"test_{i},{summary}")

# CSV 파일로 저장
with open(SUBMIT_PATH, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["fname", "summary"])
    writer.writerows(summaries)

test_0,실장님이 더슨 씨에게 내부 메모로 전달해야 하는지 물어봅니다. 더슨 씨는 실장님에게 모든 직원들에게 메모를 준비하라고 요청합니다.
test_1,"#Person2#는 교통 체증에 걸렸고, #Person1#은 #Person2#에게 대중교통을 이용하라고 조언한다. #Person2#는 #Person1#이 자전거로 출근하라고 제안한다."
test_2,#Person1#은 케이트에게 마샤와 히어로가 이혼하려고 한다고 말한다. 케이트는 놀라며 이혼이 놀라운 일이라고 생각한다.
test_3,"브라이언은 #Person1#의 생일 파티에서 춤을 추고, #Person1#은 그를 위해 술을 마시는 것을 기쁘게 생각한다."
test_4,#Person2#는 #Person1#에게 올림픽 공원의 중심인 올림픽 스타디움에 대해 설명합니다.
test_5,"#Person1#은 #Person2#에게 사업 계획서 작성에 대해 묻습니다. #Person2#는 #Person1#에게 사업 아이디어, 회사, 상품, 서비스, 시장 분석, 재무 분석, 투자자 정보 등을 포함하는 사업 계획서 작성에 대해 가르칩니다."
test_6,#Person2#는 수두에 걸렸다고 #Person1#에게 말한다. #Person1#는 #Person2#가 수두에 걸린 것 같다고 생각하고 #Person2#에게 의사를 만나라고 권한다.
test_7,#Person2#는 체크아웃하고 세탁 서비스 비용을 청구받았습니다. #Person1#는 실수로 청구서를 수정할 것입니다.
test_8,"#Person1#은 자신의 잘못을 깨달았고, 아내와 이혼하려고 합니다. 스티븐은 #Person1#에게 그녀를 설득하기 위해 최선을 다하겠다고 맹세합니다."
test_9,"#Person2#는 #Person1#에게 에이브러햄 링컨을 예로 들어, 그가 미국 대통령으로서의 성격을 소개합니다."
test_10,#Person1#은 #Person2#에게 허베이가 모래폭풍에 취약하다고 말합니다.
test_11,"프란시스는 #Person2#에게 그의 생일 파티에 

FileNotFoundError: [Errno 2] No such file or directory: '/kkh/prediction/submission_google-gemma-2b-it-02.csv'